In [2]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms

In [3]:
data_dir = '/Users/sunxiaolei/Python_Projects/Deeplearning/experiment_three'

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize((0.5, ), (0.5, ))
])

In [5]:
batch_size = 64
trainset = datasets.FashionMNIST(root=data_dir, download=True, train=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle = True)

testset = datasets.FashionMNIST(root=data_dir, download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle = True)

In [6]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size = 11, stride = 4, padding=2),
            nn.ReLU(inplace = True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [7]:
model = AlexNet()

In [8]:
model

AlexNet(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [9]:
import time
model = AlexNet()
def training(trainloader, testloader, lr, epochs):
    # model.to('cuda')

    criterion = nn.CrossEntropyLoss()

    optimizer = optim.SGD(model.parameters(), lr=0.003)

    train_losses, test_losses = [], []

    start_time = time.time()
    print("-------------训练开始：lr={}, epochs={}----------".format(lr, epochs))

    for e in range(epochs):
        running_loss = 0

        for images, labels in trainloader:

            # images = images.cuda()
            # labels = labels.cuda()

            optimizer.zero_grad()

            log_ps = model(images)
            loss = criterion(log_ps, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        else:
            test_loss = 0
            accuracy = 0

            with torch.no_grad():
                model.eval()

                for images, labels in testloader:
                    # images = images.cuda
                    # labels = labels.cuda()

                    log_ps = model(images)
                    test_loss += criterion(log_ps, labels)
                    ps = torch.exp(log_ps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor))

            model.train()

            train_losses.append(running_loss/len(trainloader))
            test_losses.append(test_loss/len(testloader))
            info = "训练集学习次数: {}/{}.. ".format(e+1,epochs),"训练误差: {:.3f}..".format(running_loss/len(trainloader)),\
            "测试误差: {:.3f}..".format(test_loss/len(testloader)),"模型分类准确率:{:.3f}".format(accuracy/len(testloader))
            print(info)
    end_time = time.time()
    run_time = end_time - start_time
    return [lr, epochs, train_losses, test_losses, run_time]

In [10]:
%time [lr, epochs, train_losses, test_losses, run_time]=training(trainloader, testloader, 0.003, 15)

-------------训练开始：lr=0.003, epochs=15----------


/Users/sunxiaolei/miniconda3/envs/pytorch/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(

KeyboardInterrupt



In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
for i in range(len(test_losses)):
    test_losses[i] = test_losses[i].cpu()

plt.plot(test_losses, label='Testing loss')
plt.plot(train_losses, label='Training loss')
plt. legend()